In [3]:
import pandas as pd
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [8]:
import re
# csv -> json
root = 'work_dir/co_detr_skfold_1st/'
csv_path = root + 'co-detr_fp16.csv'
submission_path = root + 'coco_submission.json'

submission = pd.read_csv(csv_path)

coco_submission = []

def extract_image_id(image_path):
    image_id = re.findall(r'\d+', image_path)[0]
    return int(image_id)

for idx, row in submission.iterrows():
    image_id = extract_image_id(row['image_id'])
    print(idx, row['PredictionString'])
    predictions = row['PredictionString'].split(' ')
    
    predictions = [pred for pred in predictions if pred.strip()]

    for i in range(0, len(predictions), 6):
        category_id = int(predictions[i])
        score = float(predictions[i + 1])
        bbox = [float(predictions[i + 2]), float(predictions[i + 3]), 
                float(predictions[i + 4]) - float(predictions[i + 2]), 
                float(predictions[i + 5]) - float(predictions[i + 3])] 

        prediction = {
            "image_id": image_id,
            "category_id": category_id,
            "bbox": bbox,
            "score": score
        }
        coco_submission.append(prediction)

with open(submission_path, 'w') as f:
    json.dump(coco_submission, f, indent=4)



7 0.9210 212.98 51.74 454.57 472.70 7 0.8781 603.09 520.45 956.29 1023.20 7 0.8729 388.45 190.53 599.14 546.65 7 0.8204 118.66 421.51 331.99 722.19 7 0.8093 666.91 339.02 876.86 603.40 5 0.7573 243.75 683.38 300.84 747.87 1 0.7516 564.60 104.69 739.66 358.45 7 0.7382 552.98 341.27 718.73 608.01 7 0.6919 266.10 441.33 428.68 581.36 7 0.4976 449.67 600.02 645.71 874.34 6 0.4914 431.14 468.19 560.96 675.76 5 0.4436 330.01 640.39 395.36 725.27 5 0.3884 318.58 674.60 465.83 745.51 1 0.3735 639.42 260.58 759.67 346.33 5 0.3540 392.83 616.70 453.36 687.49 5 0.3393 322.52 614.63 463.80 745.36 5 0.3286 346.22 682.84 424.45 726.79 5 0.2874 384.89 684.08 423.36 721.54 5 0.2805 329.61 565.46 451.70 726.37 7 0.2762 322.16 568.37 379.31 619.25 0 0.2683 464.13 552.22 559.98 599.55 5 0.2631 322.16 568.37 379.31 619.25 5 0.2615 334.73 640.36 423.96 726.34 7 0.2564 540.14 186.77 574.31 238.57 1 0.2497 464.13 552.22 559.98 599.55 1 0.2440 676.82 260.39 726.50 312.66 1 0.2416 480.61 612.10 508.61 659.26 5

AttributeError: 'float' object has no attribute 'split'

In [3]:
coco_gt = COCO('/data/ephemeral/home/whth/level2-objectdetection-cv-16/dataset/train.json') # gt데이터
coco_dt = coco_gt.loadRes('/data/ephemeral/home/whth/level2-objectdetection-cv-16/coco_submission.json') # json으로 바꾼거경로따오기
# coco_gt, coco_pred 선언법 걍 이게 맞다 ㅇㅋ

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.71s)
creating index...
index created!


In [4]:
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=16.14s).
Accumulating evaluation results...
DONE (t=2.20s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.648
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.828
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.700
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.033
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.307
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.727
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.368
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.668
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.709
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.098
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=10

In [5]:
metrics = {
    "category": [],
    "mAP": [],
    "mAP_50": [],
    "mAP_75": [],
    "mAP_s": [],
    "mAP_m": [],
    "mAP_l": []
}

In [6]:
for i, cat_id in enumerate(coco_gt.getCatIds()): # 각 카테고리마다
    cat_name = coco_gt.loadCats(cat_id)[0]['name']
    coco_eval.params.catIds = [cat_id] 
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    metrics["category"].append(cat_name)
    metrics["mAP"].append(coco_eval.stats[0])  
    metrics["mAP_50"].append(coco_eval.stats[1])
    metrics["mAP_75"].append(coco_eval.stats[2]) 
    metrics["mAP_s"].append(coco_eval.stats[3])  
    metrics["mAP_m"].append(coco_eval.stats[4])  
    metrics["mAP_l"].append(coco_eval.stats[5]) 

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.42s).
Accumulating evaluation results...
DONE (t=0.71s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.517
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.689
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.557
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.039
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.322
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.628
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.377
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.599
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.618
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.192
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100

In [7]:
df_metrics = pd.DataFrame(metrics)
print(df_metrics)

        category       mAP    mAP_50    mAP_75     mAP_s     mAP_m     mAP_l
0  General trash  0.517200  0.688834  0.556724  0.039041  0.321839  0.628316
1          Paper  0.504926  0.746821  0.531792  0.041264  0.216573  0.588564
2     Paper pack  0.703972  0.851964  0.758210  0.000000  0.350602  0.812032
3          Metal  0.674579  0.828272  0.720200  0.095449  0.298399  0.801319
4          Glass  0.662780  0.861371  0.721062  0.004569  0.254990  0.739140
5        Plastic  0.583616  0.770979  0.631107  0.027037  0.284367  0.694945
6      Styrofoam  0.610247  0.830038  0.651207  0.000000  0.298994  0.659417
7    Plastic bag  0.680294  0.864945  0.747102  0.011256  0.262883  0.748628
8        Battery  0.824328  0.964793  0.897701 -1.000000  0.629401  0.847937
9       Clothing  0.721406  0.871457  0.784647  0.075743  0.156300  0.753100
